In [1]:
import preamble
from virtual_dynamics import SimpleVirtualDynamics
from robot import Robot
from app_loop import AppLoop
from force_guider import SimpleForceGuider
from language_generator import DirectionalLanguageGenerator
from vocalizer import Vocalizer
import numpy as np
import random

In [2]:
class HandGuidingGame(AppLoop):

    MASS = 20
    AXES = Robot.TRANSLATION
    PHRASES = [
        [""],
        ["move <direction>"],
        ["move <direction> more"],
        ["You need to move <direction>"]
    ]
    DIRECTION_PAIRS = [("right", "left"), ("forward", "backward"), ("down", "up")]

    def __init__(self, randomize_goal: bool = False, use_guiding_force: bool = True, use_language: bool = True) -> None:
        super().__init__()
        self.use_guiding_force = use_guiding_force
        self.use_language = use_language
        self.randomize_goal = randomize_goal

    def startup(self) -> None:
        self.robot = Robot("169.254.9.43",
                           translational_force_deadband=6.0,
                           rotational_force_deadband=0.5)
        self.robot.control.zeroFtSensor()

        self.dynamics = SimpleVirtualDynamics(M=self.MASS, B=50, K=0)

        if self.randomize_goal:
            self.goal_point = np.array(
                [random.uniform(-1.0, -0.5), random.uniform(-0.6, 0.6), random.uniform(0.0, 0.6)])
        else:
            self.goal_point = self.robot.get_pose(self.AXES)

        self.force_guider = SimpleForceGuider()
        self.language_generator = DirectionalLanguageGenerator(self.PHRASES, self.DIRECTION_PAIRS, magnitude_interval=12)
        self.vocalizer = Vocalizer(buffer_period=2.0)

    def update(self, t: float, dt: float) -> None:
        period_start = self.robot.control.initPeriod()

        F_human = self.robot.get_force(self.AXES)
        velocity = self.robot.get_velocity(self.AXES)
        delta_x = self.robot.get_pose(self.AXES) - self.goal_point
        F_trajectory = -self.MASS * (velocity + 2.0 * delta_x)
        F_error = F_trajectory - F_human

        if self.use_guiding_force:
            F_guide = self.force_guider.generate_guiding_force(F_error, F_human)
        else:
            F_guide = 0.0
        
        if self.use_language:
            utterance = self.language_generator.generate(F_error)
            if np.linalg.norm(delta_x) < 0.05:
                utterance = "You are at the goal."
            self.vocalizer.utter(utterance)

        self.dynamics.apply_force(F_human + F_guide, dt)
        self.robot.set_velocity(self.dynamics.get_velocity(),
                                Robot.TRANSLATION, acceleration=150)

        self.robot.control.waitPeriod(period_start)

    def shutdown(self) -> None:
        self.robot.set_velocity(Robot.zeroed_translation_rotation())

In [ ]:
app = HandGuidingGame(randomize_goal=True, use_guiding_force=True, use_language=True)
app.run()